<a href="https://colab.research.google.com/github/atahanatik/ADM_homework1/blob/main/Scheduling_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import heapq

In [ ]:
#Upload the dataset
df= pd.read_csv(r"/content/NBD_homework_data.csv")
df=df[:500]#delete later
#Sorting the data by Arrival_Time and creating a new Task ID 
df=df.sort_values("Arrival_Time")
df['TaskNewID'] = df['Job_ID'].astype(str) +"TATA"+ df['Task_ID'].astype(str)
#X = C/µ
df["expected_execution_time_milisecond"]=df["CPU"]*1000/0.1

In [ ]:
heap=[]
is_it_in_heap= set()
status_table=   pd.DataFrame(columns=["Time", "Task_list","event"])
available_memory=0.01
current_time=df.Arrival_Time[0]
current_time_df=pd.DataFrame(columns=["TaskNewID","expected_execution_time_milisecond","Memory",])

In [ ]:
  for i,row in df.iterrows():
    if row.Arrival_Time <= current_time:
      #Creating a new dataframe for every time period    
      save_the_values=pd.DataFrame({"TaskNewID":[row.TaskNewID],
                                    "expected_execution_time_milisecond":[row.expected_execution_time_milisecond],
                                    "Memory":[row.Memory]})
      
      current_time_df = pd.concat([current_time_df, save_the_values])
      #The tasks are put into order for calculation in a heap if there is available memory
      for index, row in current_time_df.iterrows():
        if available_memory-row.Memory>0 and row['TaskNewID'] not in is_it_in_heap :
          task_finish_time = current_time + row["expected_execution_time_milisecond"]
          heapq.heappush(heap,(task_finish_time, row['TaskNewID'],row["Memory"]))
          #updating memory availability
          available_memory=available_memory-row.Memory
          #updating the status table 
          save_to_status_table=pd.DataFrame({"Time":[current_time],
                                             "Task_list":[list(heap)],
                                             "event":[f"{row.TaskNewID} is scheduled"],
                                             "used_memory":[1-available_memory]})
          status_table=pd.concat([status_table, save_to_status_table])
          is_it_in_heap.add(row['TaskNewID'])
        else:
          #wait for the next time 
          #a task is completed
          if i+1 < len(df):
            if df.Arrival_Time[i]-df.Arrival_Time[i+1]>df.expected_execution_time_milisecond[i]:
              current_time=current_time+df.expected_execution_time_milisecond[i]
              #make sure memory of the completed calculation is freed
              available_memory=available_memory+heap[0][2]
              heapq.heappop(heap)
            else:
             #check if there is any task completed
             if heap and current_time>=heap[0][0]:
               _, _, memory_used= heapq.heappop(heap)
               available_memory=available_memory+memory_used
             current_time=max(current_time, df.Arrival_Time[i+1])
          else:
            pass
    else:
      #let the time pass and when the arrival time of i become smaller than current time, include row to the current_df
      while i < len(df) and df.Arrival_Time[i] > current_time:
          if heap and current_time>=heap[0][0]:
            _, _, memory_used= heapq.heappop(heap)
            available_memory=available_memory+memory_used
          current_time = heap[0][0] if heap else df.Arrival_Time[i]

  #will add notifiers to the status table for tasks that are too big for memory

In [ ]:
status_table